In [1]:
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import requests, os, sys
import pyspark

warnings.filterwarnings('ignore')
pd.options.display.mpl_style = 'default'

%matplotlib inline

In [8]:
!head agaricus-lepiota.data

p,x,s,n,t,p,f,c,n,k,e,e,s,s,w,w,p,w,o,p,k,s,u
e,x,s,y,t,a,f,c,b,k,e,c,s,s,w,w,p,w,o,p,n,n,g
e,b,s,w,t,l,f,c,b,n,e,c,s,s,w,w,p,w,o,p,n,n,m
p,x,y,w,t,p,f,c,n,n,e,e,s,s,w,w,p,w,o,p,k,s,u
e,x,s,g,f,n,f,w,b,k,t,e,s,s,w,w,p,w,o,e,n,a,g
e,x,y,y,t,a,f,c,b,n,e,c,s,s,w,w,p,w,o,p,k,n,g
e,b,s,w,t,a,f,c,b,g,e,c,s,s,w,w,p,w,o,p,k,n,m
e,b,y,w,t,l,f,c,b,n,e,c,s,s,w,w,p,w,o,p,n,s,m
p,x,y,w,t,p,f,c,n,p,e,e,s,s,w,w,p,w,o,p,k,v,g
e,b,s,y,t,a,f,c,b,g,e,c,s,s,w,w,p,w,o,p,k,s,m


In [21]:
df=pd.read_csv('agaricus-lepiota.data',header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [3]:
#1)  get data from UCI using wget or scp.. 
# https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.names

spark=pyspark.sql.SparkSession.builder.getOrCreate()

In [42]:
#2)  read in the data with spark DataFrame
data=spark.read.csv('agaricus-lepiota.data',
                   sep=',',
                   inferSchema=True,
                   header=False)

#data.printSchema()


In [43]:
y=data[['_c0']]
data=data.drop('_c0')

In [44]:
columns=["cap_shape","cap_surface","cap-color", "bruises","odor","gill_attachment",
         "gill_spacing","gill_size","gill_color","stalk_shape","stalk-root","stalk_surface_above_ring",
        "stalk_surface_below_ring","stalk_color_above_ring",
         "stalk_color_below_ring","veil_type","veil_color","ring_number","ring_type",
         "spore_print_color","population","habitat"]
         
old_columns=data.schema.names

for old,new in zip(old_columns,columns):
    data=data.withColumnRenamed(old,new)

In [45]:
def preview(df, n=5):
    return pd.DataFrame(df.take(n), columns=df.columns)

preview(data)

,cap_shape,cap_surface,cap-color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,stalk_shape,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,x,s,n,t,p,f,c,n,k,e,...,s,w,w,p,w,o,p,k,s,u
1,x,s,y,t,a,f,c,b,k,e,...,s,w,w,p,w,o,p,n,n,g
2,b,s,w,t,l,f,c,b,n,e,...,s,w,w,p,w,o,p,n,n,m
3,x,y,w,t,p,f,c,n,n,e,...,s,w,w,p,w,o,p,k,s,u
4,x,s,g,f,n,f,w,b,k,t,...,s,w,w,p,w,o,e,n,a,g


In [78]:
# we have a heck of lot of dummy variables for this dataset ! 
# see https://spark.apache.org/docs/2.1.0/ml-features.html#stringindexer

#3) Transform data to dummy variables .. Use pipeline implemenation
# Give ? Hint ?  : String Indexer + OneHotEncoder.. 

# you might have to google around for a good solution on how to transform many 
# columns at once, 

from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

#si =StringIndexer(inputCol='_c0',outputCol='c0_float')
#si_model=si.fit(df)
#df2=si_model.transform(df)

indexers=[StringIndexer(inputCol=column, outputCol=column+"_index").fit(data) for column in data.columns]

In [79]:
pipeline=Pipeline(stages=indexers)
data_tr=pipeline.fit(data).transform(data)

In [84]:
# cleaning up.. 
for column in data.columns:
    data_tr=data_tr.drop(column)

In [85]:
preview(data_tr)  # we have acquired labels

,cap_shape_index,cap_surface_index,cap-color_index,bruises_index,odor_index,gill_attachment_index,gill_spacing_index,gill_size_index,gill_color_index,stalk_shape_index,...,stalk_surface_below_ring_index,stalk_color_above_ring_index,stalk_color_below_ring_index,veil_type_index,veil_color_index,ring_number_index,ring_type_index,spore_print_color_index,population_index,habitat_index
0,0.0,1.0,0.0,1.0,6.0,0.0,0.0,1.0,7.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,4.0
1,0.0,1.0,3.0,1.0,4.0,0.0,0.0,0.0,7.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0
2,3.0,1.0,4.0,1.0,5.0,0.0,0.0,0.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,5.0
3,0.0,0.0,4.0,1.0,6.0,0.0,0.0,1.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,4.0
4,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,4.0,1.0


In [107]:
# strange error on veil_type_index.. only one type of label here.. 

cnts=spark.sql("""select AVG(veil_type_index) as ct from data_dr""")

cnts.show()

+---+
| ct|
+---+
|0.0|
+---+



In [113]:
veil_type.columns

['veil_type']

In [ ]:
# isolate veil_type_index for now, we can bring that back later .. 

veil_type=data[['veil_type']]
data=data.drop('veil_type')

In [94]:
from pyspark.ml.feature import OneHotEncoder

onehotencoder_stages=[OneHotEncoder(inputCol=column+"_index",outputCol=column+'_ohe') for column in data.columns]
stages=indexers + onehotencoder_stages

pipeline=Pipeline(stages=stages)
data_tr=pipeline.fit(data).transform(data)

In [105]:
from pyspark.ml.feature import VectorAssembler

features=data_tr.columns[-21:]

vectorassembler = VectorAssembler(inputCols=features, outputCol='feature_vect')

data_final=vectorassembler.transform(data_tr)
data_final=data_final[['feature_vect']]

In [106]:

preview(data_final)

,feature_vect
0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, ..."
1,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
2,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
3,"(1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
4,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
